In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
class CNN(nn.Module):
    def __init__(self, in_channel, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channel, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

In [3]:
x = torch.randn(64, 1, 28, 28)
model = CNN(in_channel = x.shape[1], num_classes=10)
model(x).shape

torch.Size([64, 10])

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
batch = 64
input_size = 784
output_size = 10
learning_rate = 0.001
num_epochs = 5

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch, shuffle=True)

In [7]:
model = CNN(in_channel = 1, num_classes = 10).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    with tqdm(train_loader) as tqdm_epoch:
        for images, labels in tqdm_epoch:
            tqdm_epoch.set_description(f"Epoch {epoch + 1}/{num_epochs}")

            data = images.to(device)
            targets = labels.to(device)
        
            scores = model(data)
            loss = criterion(scores, targets)
        
            optimizer.zero_grad()
            loss.backward()
        
            optimizer.step()
            tqdm_epoch.set_postfix(loss=loss.item())

Epoch 5/5: 100%|████████████████████████████████████████████████████████| 938/938 [00:16<00:00, 55.76it/s, loss=0.0337]


In [12]:
def check_accuracy(loader, model):
    num_correct = 0
    num_sample = 0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
        
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_sample += predictions.size(0)
        
        print(f'{num_correct} / {num_sample} with accuracy {float(num_correct)/float(num_sample)*100:2f}')
    
    model.train()

In [13]:
check_accuracy(train_loader, model)

59112 / 60000 with accuracy 98.520000


In [14]:
check_accuracy(test_loader, model)

9840 / 10000 with accuracy 98.400000
